In [5]:
import requests
import json
import pandas as pd
import snowflake.connector
from sqlalchemy import create_engine
import sqlalchemy as db
import mysql.connector
import requests
import time
from email.message import EmailMessage
import ssl
import smtplib
from snowflake.connector.pandas_tools import write_pandas
import pywhatkit
import webbrowser
import pyautogui
from time import sleep
from datetime import datetime

In [6]:
api_key = "INGRESAR TU API"

Este codigo descarga el catalogo de tipo de monedas de la API.

In [7]:
url = f"https://api.currencybeacon.com/v1/currencies?api_key={api_key}"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    data = data['response']['fiats']
    df = pd.DataFrame(data).T
else:
    print("Error en la solicitud:", response.status_code)
df

,currency_name,currency_code,decimal_units,countries
AED,United Arab Emirates dirham,AED,2,[United Arab Emirates]
AFN,Afghan afghani,AFN,2,[Afghanistan]
ALL,Albanian lek,ALL,2,[Albania]
AMD,Armenian dram,AMD,2,[Armenia]
ANG,Netherlands Antillean guilder,ANG,2,"[Curaçao (CW), Sint Maarten (SX)]"
...,...,...,...,...
XXX,No currency,XXX,.,[]
YER,Yemeni rial,YER,2,[Yemen]
ZAR,South African rand,ZAR,2,"[Lesotho, Namibia, South Africa]"
ZMW,Zambian kwacha,ZMW,2,[Zambia]


In [8]:
catalogo  = pd.DataFrame(df)

Este codigo descarga los valores de junio por moneda y por dia.

In [9]:
data_june = []

for n in range(1,31):
    date = f'2023-06-{str(n).zfill(2)}'

    url = f"https://api.currencybeacon.com/v1/historical?api_key={api_key}&date={date}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        data_june.append(data)
    else:
        print("Error en la solicitud:", response.status_code)

In [10]:
df_tipos_cambios = pd.DataFrame(columns=['Moneda', 'Valor', 'Fecha'])  # Crear el DF donde se almacenarán los datos

for data in data_june:
    fecha = data['response']['date']
    rates = data['response']['rates']

    datos_moneda = pd.DataFrame({'Moneda': list(rates.keys()), 'Valor': list(rates.values()), 'Fecha': fecha})
    df_tipos_cambios = pd.concat([df_tipos_cambios, datos_moneda], ignore_index=True)
df_tipos_cambios

,Moneda,Valor,Fecha
0,ADA,2.703578,2023-06-01
1,AED,3.672500,2023-06-01
2,AFN,87.157352,2023-06-01
3,ALL,101.517051,2023-06-01
4,AMD,383.885553,2023-06-01
...,...,...,...
6595,ZAR,18.835641,2023-06-30
6596,ZMK,17716.909606,2023-06-30
6597,ZMW,17.716910,2023-06-30
6598,ZWD,361.900000,2023-06-30


In [11]:
datos_junio = pd.DataFrame(df_tipos_cambios)

Estos son los valores maximos de junio por moneda.

In [12]:
# Convertir la columna 'Valor' a tipo numérico
df_tipos_cambios['Valor'] = pd.to_numeric(df_tipos_cambios['Valor'])

# Encontrar el día con la mayor tasa de cambio para cada moneda
max_dates = df_tipos_cambios.groupby('Moneda')['Fecha'].max()
max_values = df_tipos_cambios.groupby(['Moneda', 'Fecha'])['Valor'].max().reset_index()

# Combinar los resultados en un nuevo DataFrame
result_df = pd.merge(max_dates, max_values, on=['Moneda', 'Fecha'])

# Imprimir el resultado
print(result_df)


    Moneda       Fecha         Valor
0      ADA  2023-06-30      3.484087
1      AED  2023-06-30      3.672500
2      AFN  2023-06-30     86.043061
3      ALL  2023-06-30     97.487004
4      AMD  2023-06-30    387.085317
..     ...         ...           ...
215    ZAR  2023-06-30     18.835641
216    ZMK  2023-06-30  17716.909606
217    ZMW  2023-06-30     17.716910
218    ZWD  2023-06-30    361.900000
219    ZWL  2023-06-30   5738.528023

[220 rows x 3 columns]


In [13]:
resultado = pd.DataFrame(result_df)
result_df.to_csv('max_tasas_cambio.csv', index=False)

El siguiente codigo manda el archivo max_tasas_cambio.csv por mail.

In [14]:
#este envia un archivo

email_emisor = 'INGRESAR MAIL EMISOR'   
email_contraseña = 'CLAVE DE GOOGLE' 
email_receptor = 'INGRESAR EL MAIL RECEPTOR'

asunto = 'mes de junio'
cuerpo = ''' 
estos son los tipos de cambio de todo junio
'''

em = EmailMessage()
em['From'] = email_emisor
em['To'] = email_receptor
em['Subject'] = asunto
em.set_content(cuerpo)

# Ruta y nombre del archivo adjunto
ruta_archivo = 'max_tasas_cambio.csv'

# Leer el archivo y agregarlo como adjunto
with open(ruta_archivo, 'rb') as archivo:
    contenido_adjunto = archivo.read()
    em.add_attachment(contenido_adjunto, maintype='application', subtype='octet-stream', filename=ruta_archivo)

contexto = ssl.create_default_context()

with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=contexto) as smtp:
    smtp.login(email_emisor, email_contraseña)
    smtp.sendmail(email_emisor, email_receptor, em.as_string())


Este codigo realiza la conexion a Snowflake, crea las tablas e ingresa los datos descargados de la API.

In [15]:

# Create the connection to the Snowflake database.
conn = snowflake.connector.connect(
    user='TU USUARIO',
    password='CONTRASEÑA',
    account='REGION',
    warehouse='WAREHOUSE USADO',
    database='DB',
    schema='SCHEMA'
)

#Escribir los datos del DataFrame RESULTANTES en una nueva tabla PONERLE NOMBRE"
success, nchunks, nrows, _ = write_pandas(conn, catalogo , 'catalogo_monedas', auto_create_table=True)
success, nchunks, nrows, _ = write_pandas(conn, datos_junio, 'datos_junio', auto_create_table=True)
success, nchunks, nrows, _ = write_pandas(conn, resultado, 'maximo_cambio', auto_create_table=True)


TASAS EN TIEMPO REAL EJECUTADO CINCO VECES UNA VEZ POR HORA (esto tiene que ir al final porque queda en espera de ejecucion)

In [16]:


# Configuración del correo electrónico
email_emisor = 'INGRESA TU EMAIL'
email_contraseña = 'CONTRASEÑA GOOGLE'
email_receptor = 'EMAIL RECEPTOR'

# Función para enviar el correo electrónico
def enviar_correo(asunto, cuerpo):
    em = EmailMessage()
    em['From'] = email_emisor
    em['To'] = email_receptor
    em['Subject'] = asunto
    em.set_content(cuerpo)

    contexto = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=contexto) as smtp:
        smtp.login(email_emisor, email_contraseña)
        smtp.sendmail(email_emisor, email_receptor, em.as_string())

# Configuración de Snowflake
conn = snowflake.connector.connect(
    user='TU USUARIO',
    password='CONTRASEÑA',
    account='REGION',
    warehouse='WAREHOUSE USADO',
    database='DB',
    schema='SCHEMA'
)

url = f"https://api.currencybeacon.com/v1/latest?api_key={api_key}"
params = {"base": "USD"}

# Variables para almacenar los datos
datos_tiempo_real = []

# Ejecutar el código cinco veces
for _ in range(5):
    # Realizar la solicitud GET a la API
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        df_tiempo_real = pd.DataFrame(columns=['Divisas', 'Valores', 'Fecha', 'Base'])

        try:
            fecha = data['response']['date']
            divisas = data['response']['rates'].keys()
            rates = data['response']['rates']

            df_tiempo_real['Divisas'] = divisas

            lst_divisas = df_tiempo_real['Divisas']
            for e in lst_divisas:
                if e not in df_tiempo_real['Divisas'].values:
                    nuevo_registro = pd.Series({'Divisas': e})
                    df_tiempo_real = df_tiempo_real.append(nuevo_registro, ignore_index=True)

            for divisa, tasa in rates.items():
                df_tiempo_real.loc[df_tiempo_real['Divisas'] == divisa, 'Valores'] = tasa

            df_tiempo_real['Fecha'] = fecha
            df_tiempo_real['Base'] = 'USD'

            # Agregar los datos al listado
            datos_tiempo_real.append(df_tiempo_real)

        except:
            pass

    else:
        print("Error en la solicitud:", response.status_code)

   

# Imprimir el último DataFrame resultante
print(datos_tiempo_real[-1])

# Enviar correo electrónico
enviar_correo('Datos actualizados', 'Se han obtenido nuevos datos de la API.')

# Combinar todos los DataFrames en uno solo
df_completo = pd.concat(datos_tiempo_real)

datos_actualizados =pd.DataFrame (df_completo)

# Cargar datos en Snowflake
success, nchunks, nrows, _ = write_pandas(conn, datos_actualizados , 'datos_actualizados', auto_create_table=True)

# Esperar una hora antes de la siguiente ejecución
time.sleep(3600)

    Divisas       Valores                 Fecha Base
0       ADA      3.364356  2023-07-03T19:46:50Z  USD
1       AED        3.6725  2023-07-03T19:46:50Z  USD
2       AFN     85.956995  2023-07-03T19:46:50Z  USD
3       ALL     96.977972  2023-07-03T19:46:50Z  USD
4       AMD    384.413441  2023-07-03T19:46:50Z  USD
..      ...           ...                   ...  ...
215     ZAR     18.758659  2023-07-03T19:46:50Z  USD
216     ZMK  17781.832741  2023-07-03T19:46:50Z  USD
217     ZMW     17.781833  2023-07-03T19:46:50Z  USD
218     ZWD         361.9  2023-07-03T19:46:50Z  USD
219     ZWL   5735.482238  2023-07-03T19:46:50Z  USD

[220 rows x 4 columns]


Este codigo manda un whatsapp avisando que el codigo se ejecuto correctamente. 

In [ ]:
pywhatkit.sendwhatmsg("aca ingresar el numero" , " Mensaje" , hs , min )

from time import sleep

webbrowser.open ('https://web.whatsapp.com/send?phone=[aca ingresar el numero]')

sleep(3000)

for i in range(2):
    pyautogui.typewrite('esto funciona')
    pyautogui.press('enter')
    sleep(2)


El siguiente codigo crea el procedimiento y la vista en mysql. Fue creado en un testeo y luego no me dejo borrarlo en Mysql, denego el acceso a mi usuario. Asique no sera ejecutado (pero esta)

In [ ]:
'''# Conectar a la base de datos
connection = mysql.connector.connect(
    host="",
    user="",
    password="*",
    database=""
)

# Crear un cursor para ejecutar consultas
cursor = connection.cursor()

# Definir el código del procedimiento almacenado
procedimiento_codigo = """
CREATE PROCEDURE ObtenerHistoricoTasasDeCambio()
BEGIN
    -- Crea una vista temporal para almacenar el resultado final
    CREATE TEMPORARY TABLE TempHistoricoTasasDeCambio AS (
        SELECT H.Fecha, H.Moneda, H.Valor
        FROM datos_junio H
        ORDER BY H.Fecha DESC
    );

    -- Realiza la unión con la tabla de catálogo de monedas
    SELECT T.Fecha, T.Moneda, T.Valor, C.currency_name, C.currency_code, C.decimal_units, C.countries
    FROM TempHistoricoTasasDeCambio T
    LEFT JOIN catalogo_monedas C ON T.Moneda = C.currency_code;
END;
"""

try:
    # Ejecutar la consulta para crear el procedimiento almacenado
    cursor.execute(procedimiento_codigo)

    # Confirmar los cambios en la base de datos
    connection.commit()

    print("Procedimiento almacenado creado exitosamente.")
except mysql.connector.Error as error:
    print(f"Error al crear el procedimiento almacenado: {error}")

# Cerrar la conexión
cursor.close()
connection.close()
'''